# Exploring NumPy's Issues with the GitHub GraphQL API

Like most open source projects, NumPy has many open issues and pull requests that require developer attention, and not nearly enough developer-hours to address them all. Even the simple task of reading through the issues/PRs would take hours as there are on the order of 1000 open issues and ~200 open PR's for NumPy (as of 1/2020). Thus it might be worthwhile to attempt some sort of analysis to determine the relative *importance* of the open issues. 

A simple idea would be to look at the number of times any given issue has been cross-referenced as a weak proxy for *importance*, i.e. how often is a particular concept referenced by other developers. A question like this seems like a good match for GitHub's new [GraphQL API](https://developer.github.com/v4/guides/intro-to-graphql/). Note the API is not really about building graphs but rather a different way of formulating queries of GitHub's data. Nevertheless the GraphQL API seems to be well-suited for the type of question we are interested in asking, as it is natural to think of issues as *nodes* and cross-references as *edges*.

## Getting started with GraphQL

I had never heard about GraphQL (and am certainly no expert in database querying), but I found the [Intro to GitHub's GraphQL API](https://developer.github.com/v4/guides/intro-to-graphql/) to be very accessible. 

The [Github GraphQL Explorer](https://developer.github.com/v4/explorer/) is super useful for building your own queries.

## The question

It helps to have a clear statement of the question we want answered. The simplest statement of our question would be

> *How many times has each open issue in the numpy repo been cross-referenced?*

If we cast this question in terms of our simple graph model (open issues as *nodes* and cross-references as *edges*), it is clear that we can find the answer by counting the number of edges connected to each node!

Of course there are a ton of ways that we could modify this question to make it more specific, necessitating a more complicated query to answer it. We'll get there, but let's start simple.

## Formulating a query to address the question

I started by loading the [sample query](https://developer.github.com/v4/guides/forming-calls/#example-query) into the explorer and slowly modifying the query by looking up features in the [schema reference](https://developer.github.com/v4/). This eventually crystallized into the following query:

```
query {                                                                        
  repository(owner:"numpy", name:"numpy") {                                     
    issues(first:100 states:OPEN) {                                             
      edges {                                                                   
        node {
          number
          timelineItems(first:100, itemTypes:CROSS_REFERENCED_EVENT){            
            totalCount                                                          
          }                                                                     
        }                                                                       
      }                                                                         
    }                                                                           
  }                                                                             
}
```

Paste the above into the [explorer](https://developer.github.com/v4/explorer/) to get proper syntax highlighting.

Building the graph of issues is [covered in the original example](https://developer.github.com/v4/guides/forming-calls/#example-query), so I'll only elaborate on the part that's different: the timelineItems.

Note that `issues` is of type [IssueConnection](https://developer.github.com/v4/object/issueconnection/) which naturally terminates on nodes of type [Issue](https://developer.github.com/v4/object/issue/). The Issue object stores the issue number in `number` and has a [timelineItems](https://developer.github.com/v4/object/issuetimelineitemsconnection/) connection that lists all the edges to events (commits, comments, references, etc.) associated with that issue. Furthermore, the `timelineItems` connection implements a filter on the [itemType](https://developer.github.com/v4/enum/issuetimelineitemsitemtype/). This allows us to only follow along connections to nodes that represent cross-reference events (as opposed to commits, comments, etc.).

Finally, the GraphQL query must terminate in scalar values. Fortunately, the `timelineItems` connection implements a `totalCount` attribute that gives us an integer value of the total number of connections. In conjunction with the `itemTypes:CROSS_REFERENCED_EVENTS` filter, this value represents the total number of cross-reference events for each issue.

## Pagination

The previous query only returns (up to) the first 100 issues that it finds that meet our criteria. This is an intentional limitation imposed by GitHub's GraphQL schema validation (Don't believe me? Try changing the `first:100` to `first:101` for the `issues` connection). This is a problem for us because we want data on all the open issues in the repo, not just the first or last 100.

I had to dig a little deeper to find out how to get past this restriction. I found an answer in the [GraphQL docs on pagination](https://graphql.org/learn/pagination/). Essentially, a cursor is provided to allow us to page through results, so that we can start the next query at a point where the result from the previous query left off.

To use this feature, a few small modifications to the previous query:

```
query {                                                                        
  repository(owner:"numpy", name:"numpy") {                                     
    issues(first:100 states:OPEN) {
      totalCount
      edges {
        cursor
        node {
          number
          timelineItems(first:100, itemTypes:CROSS_REFERENCED_EVENT){            
            totalCount                                                          
          }                                                                     
        }                                                                       
      }                                                                         
    }                                                                           
  }                                                                             
}
```

First, we ask for the `totalCount` of the number of issues. This is not required for pagination, but is useful information for us to have to make sure we're getting all the data.
We also ask for the `cursor` associated with each `IssueEdgeConnection` in issues. We can then use the cursor value from the last value in each response as input for the next query.
For example, running the above query in the explorer gives me the following result (N.B. you may see a different result when running the query in the explorer because the explorer queries real data!):

```
{
  "data": {
    "repository": {
      "issues": {
        "totalCount": 1735,
        "edges": [
          {
            "cursor": "Y3Vyc29yOnYyOpHOAFoQ1w==",
            "node": {
              "timelineItems": {
                "totalCount": 3
              }
            }
          },
          {
            "cursor": "Y3Vyc29yOnYyOpHOAGBBGg==",
            "node": {
              "timelineItems": {
                "totalCount": 0
              }
            }
          },
          
          ... 96 MORE ENTRIES REMOVED FOR BREVITY
          
          {
            "cursor": "Y3Vyc29yOnYyOpHOAHX6Mw==",
            "node": {
              "timelineItems": {
                "totalCount": 4
              }
            }
          },
          {
            "cursor": "Y3Vyc29yOnYyOpHOAHX6NA==",
            "node": {
              "timelineItems": {
                "totalCount": 0
              }
            }
          }
        ]
      }
    }
  }
}
```

The cursor associated with the last edge is `"Y3Vyc29yOnYyOpHOAHX6NA=="`. We can now use this value with the `after` parameter so that our next query starts off where the previous one ended:

```
query {                                                                        
  repository(owner:"numpy", name:"numpy", after:"Y3Vyc29yOnYyOpHOAHX6NA==") {                                     
    issues(first:100 states:OPEN) {
      totalCount
      edges {
        cursor
        node {
          number
          timelineItems(first:100, itemTypes:CROSS_REFERENCED_EVENT){            
            totalCount                                                          
          }                                                                     
        }                                                                       
      }                                                                         
    }                                                                           
  }                                                                             
}
```

## Finally - let's answer our question

Okay, so now we know we can cycle through results using `cursor` and `after`. It would be really tedious to have to modify our queries by hand in the explorer.

I hacked (emphasis on *hacked*) something to make this a little bit easier for our issue queries.
Basically, it allows copy queries from the explorer, save them as a file (the query we used above is in `query_examples/totalCount_openIssue_crossrefs.gql`) and automates the pagination through all of the issues.
Though ugly, the code in `issues.py` is mercifully short - hopefully it is grok-able.

In [ ]:
# Reminder: define the environment variable GRAPH_API_KEY with your personal github token
import issues

In [ ]:
data = issues.get_all_responses('query_examples/totalCount_openIssue_crossrefs.gql')

`get_all_responses` parses out the results for each of the issue nodes:

In [ ]:
data[:10]

In [ ]:
# A little manual munging
ndata = {
    n['node']['number'] : n['node']['timelineItems']['totalCount'] for n in data
}

In [ ]:
# Top 10 most cross-referenced issues
for node in sorted(ndata, key=ndata.get, reverse=True)[:10]:
    print("Issue {} : {} references".format(node, ndata[node]))

## Extracting more info

Our query above extracts the absolute minimum amount of information to answer our question, allowing us to list the issue number sorted by the number of crossreferences as we did in the previous cell. Now, it would be nice if we had some additional information about the issue along with it's number (e.g. the tissue title and it's URL) so we could build a more informative list. Fortunately, since we already have a valid query, this is as simple as requesting more attributes of the issue nodes:

```
query {                                                                        
  repository(owner:"numpy", name:"numpy") {                                     
    issues(first:100 states:OPEN) {
      totalCount
      edges {
        cursor
        node {
          number
          title
          createdAt
          url
          timelineItems(first:100, itemTypes:CROSS_REFERENCED_EVENT){            
            totalCount                                                          
          }                                                                     
        }                                                                       
      }                                                                         
    }                                                                           
  }                                                                             
}
```

The only difference between this query and the previous is that we've requested additional information from each issue node: `title`, `createdAt` and `url`. We can now use our pager from the `issues` module to get all the data with these additional fields. This query lives in `query_examples/totalCount_openIssue_crossrefs_withIssueData.gql`.

In [ ]:
data = issues.get_all_responses('query_examples/totalCount_openIssue_crossrefs_withIssueData.gql')

We can modify our parsing from before to keep track of the new issue attributes:

In [ ]:
ndata = {
    n['node']['number'] : {
                           "title" : n['node']['title'],
                           "url" : n['node']['url'],
                           "numrefs" : n['node']['timelineItems']['totalCount'],
                           "labels" : [lbl['node']['name'] for lbl in n['node']['labels']['edges']]
                          } for n in data
}

In [ ]:
ndata = issues.filter_issues_by_label(ndata)

And generate markdown-formatted results for nice display:

In [ ]:
from IPython.display import Markdown

In [ ]:
num_issues_to_print = 5
mdtable =  '|Iss. \#| xrefs | Issue |\n'
mdtable += '|:--------:|:----------|:-------|\n'
for node in sorted(ndata.items(), key=lambda x: x[1]['numrefs'], reverse=True)[:num_issues_to_print]:
    mdtable += "| {} | {} | [{}]({})\n".format(
        node[0],
        node[1]['numrefs'],
        node[1]['title'],
        node[1]['url']
    )
Markdown(mdtable)

# Save to file
with open("top_issues_table.md", "w") as fh:
    fh.write(mdtable)

The table-generation functionality (with some additional features) can be found in the `issues.generate_top_issues_summary` function.

In [ ]:
Markdown(issues.generate_top_issues_summary(data=data, num_issues=5))